In [10]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.metrics import r2_score 
import numpy as np 


In [11]:

conc = pd.read_csv(r"..\Cases\Concrete Strength\Concrete_Data.csv")
X = conc.drop('Strength', axis=1)
y = conc['Strength']


In [12]:

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.3, 
                                                    random_state=24)
lasso = Lasso()
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(r2_score(y_test, y_pred))


0.5763335877871639


In [13]:

########## K-FOLD #############
kfold = KFold(n_splits=5, shuffle=True, 
              random_state=24)
lambdas = np.linspace(0.001, 100,40)
scores = []
for i in lambdas:
    lasso = Lasso(alpha=i)
    results = cross_val_score(lasso, X, y,
                              cv=kfold)
    scores.append(results.mean())

i_max = np.argmax(scores)
print("Best alpha =", lambdas[i_max])


Best alpha = 0.001


In [14]:
############################

from sklearn.model_selection import GridSearchCV
params = {'alpha':np.linspace(0.001, 100,40)}
gcv = GridSearchCV(lasso, param_grid=params,
                   cv=kfold)
gcv.fit(X, y)
print(gcv.best_params_)
print(gcv.best_score_)

{'alpha': 0.001}
0.6066064857714589


In [16]:
elastic = ElasticNet()
from sklearn.model_selection import GridSearchCV

print(elastic.get_params())
params = {'alpha':np.linspace(0.001, 50,5),
          'l1_ratio':np.linspace(0.001,1, 10)}
gcv = GridSearchCV(elastic, param_grid=params,
                   cv=kfold, scoring='r2')
gcv.fit(X, y)
pd_cv = pd.DataFrame(gcv.cv_results_)
print(gcv.best_params_)
print(gcv.best_score_)

elastic.fit(X,y)
print(elastic.coef_)

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.5, 'max_iter': 1000, 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
{'alpha': 0.001, 'l1_ratio': 0.001}
0.6066065609610952
[ 0.11910483  0.10294333  0.08763564 -0.16031911  0.24967948  0.01594138
  0.01870292  0.11397284]
